# Running CG

In this notebook we show how to load and use our CG implementation.

In [1]:
import numpy as np
import torch
from core.gpytorch_log_cg_re import linear_log_cg_re

/home/pop/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


There are different variantes of the CG solver. Based on their name you can choose variantes with re-orthogonalization and with or without log-scaling.

In [2]:
N = int(1.e2)
dtype = torch.float32
A = np.random.normal(size=(N, N))
A = A @ A.T + 100
condition_number = np.linalg.cond(A)
A = torch.tensor(A, dtype=dtype)
A_fn = lambda x: A @ x
b = np.random.normal(size=(N, 1))
b = torch.tensor(b, dtype=dtype)

We created a random symmetric matrix and added with the following condition number

In [3]:
print(f'Condition of system: {condition_number:1.3e}')

Condition of system: 2.396e+06


We first solve the system using the native torch linear solver

In [4]:
check = torch.linalg.solve(A, b)

Then we use our own

In [5]:
sol = linear_log_cg_re(A_fn, b, initial_guess=torch.zeros_like(b), max_iter=int(1.e3), tolerance=1.e-2)

The relative different to the solution is

In [6]:
diff = torch.linalg.norm(check - sol) / torch.linalg.norm(sol)
print(f'Diff {diff:1.3e}')

Diff 8.689e-04
